In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import ELU, PReLU

C:\Users\anilo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df=pd.read_csv(r'E:\Havelsan\Datasets\M0000_train_sensors.csv', engine='c')
SeaTemp='SW20'
WindSpeed='WC0'
Load='FAU'
df=df[df['SeaTemp']==SeaTemp]
df=df[df['WindSpeed']==WindSpeed]
df=df[df['Load']==Load]
result=df
result.dropna(inplace=True)
result.reset_index(drop=True,inplace=True)
abr_sensor_list = ['E02005', 'E02006', 'E02056', 'E03760', 'G00027', 'G00108', 'G02011', 'N02015', 'P00023', 'P01005', 'P01302', 'P01303', 'P01600', 'P01602', 'P02055', 'P02065', 'P02066', 'P02071', 'P02072', 'Q02004', 'T00002', 'T01010', 'T01011', 'T01350', 'T01351', 'T01601', 'T01603', 'T02014', 'T02040', 'T02041', 'T02042', 'T02044', 'T04600', 'Z00518', 'Z01970', 'Z02013']
df_sample=result[abr_sensor_list]
x_train = df_sample.drop('Z02013', axis = 1) # Features
y_train = df_sample['Z02013']  # Target

In [3]:
dft=pd.read_csv(r'E:\Havelsan\Datasets\M0000_test_sensors.csv', engine='c')
dft=dft[dft['SeaTemp']==SeaTemp]
dft=dft[dft['WindSpeed']==WindSpeed]
dft=dft[dft['Load']==Load]
dft.dropna(inplace=True)
dft.reset_index(drop=True,inplace=True)
abr_sensor_list = ['E02005', 'E02006', 'E02056', 'E03760', 'G00027', 'G00108', 'G02011', 'N02015', 'P00023', 'P01005', 'P01302', 'P01303', 'P01600', 'P01602', 'P02055', 'P02065', 'P02066', 'P02071', 'P02072', 'Q02004', 'T00002', 'T01010', 'T01011', 'T01350', 'T01351', 'T01601', 'T01603', 'T02014', 'T02040', 'T02041', 'T02042', 'T02044', 'T04600', 'Z00518', 'Z01970', 'Z02013']
dft=dft[abr_sensor_list]
x_test = dft.drop('Z02013', axis = 1) # Features
y_test = dft['Z02013']  # Target

In [4]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(x_train)

X_train_scaled = scaler.transform(x_train)
X_test_scaled = scaler.transform(x_test)

In [5]:
scaler=StandardScaler()
scaler.fit(y_train.values.reshape(-1, 1))
Y_train_scaled = scaler.transform(y_train.values.reshape(-1, 1))

# LeakyReLU MAE

In [6]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 320)               11520     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 320)               0         
                                                                 
 batch_normalization (BatchN  (None, 320)              1280      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 320)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               41088     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128)               0         
                                                        

In [7]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 6s 4ms/step - loss: 0.5054 - mae: 0.4204 - val_loss: 0.2388 - val_mae: 0.2391
Epoch 2/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.2801 - mae: 0.3193 - val_loss: 0.1827 - val_mae: 0.2245
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2223 - mae: 0.2972 - val_loss: 0.1181 - val_mae: 0.1919
Epoch 4/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1879 - mae: 0.2812 - val_loss: 0.0833 - val_mae: 0.1836
Epoch 5/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1872 - mae: 0.2813 - val_loss: 0.0960 - val_mae: 0.1702
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1615 - mae: 0.2670 - val_loss: 0.0931 - val_mae: 0.1873
Epoch 7/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1666 - mae: 0.2705 - val_loss: 0.0673 - val_mae: 0.1579
Epoch 8/100
1109/1109 [==============================] 

In [8]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [9]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.009297237928971788


# LeakyReLU MSE

In [10]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 320)               11520     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 320)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 320)              1280      
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 320)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               41088     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 128)               0         
                                                      

In [11]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 6s 4ms/step - loss: 0.4846 - mse: 0.4846 - val_loss: 0.3179 - val_mse: 0.3179
Epoch 2/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2757 - mse: 0.2757 - val_loss: 0.2280 - val_mse: 0.2280
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2267 - mse: 0.2267 - val_loss: 0.1482 - val_mse: 0.1482
Epoch 4/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1932 - mse: 0.1932 - val_loss: 0.0583 - val_mse: 0.0583
Epoch 5/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1777 - mse: 0.1777 - val_loss: 0.0997 - val_mse: 0.0997
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1728 - mse: 0.1728 - val_loss: 0.0893 - val_mse: 0.0893
Epoch 7/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1536 - mse: 0.1536 - val_loss: 0.0782 - val_mse: 0.0782
Epoch 8/100
1109/1109 [==============================] 

In [12]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [13]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.007428132524179205


# ReLU MAE

In [14]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 320)               11520     
                                                                 
 activation (Activation)     (None, 320)               0         
                                                                 
 batch_normalization_6 (Batc  (None, 320)              1280      
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 320)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               41088     
                                                                 
 activation_1 (Activation)   (None, 128)               0         
                                                      

In [15]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 6s 4ms/step - loss: 0.5246 - mae: 0.4310 - val_loss: 0.1930 - val_mae: 0.2514
Epoch 2/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2840 - mae: 0.3097 - val_loss: 0.1263 - val_mae: 0.2158
Epoch 3/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.2425 - mae: 0.2913 - val_loss: 0.1015 - val_mae: 0.1987
Epoch 4/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2045 - mae: 0.2818 - val_loss: 0.0811 - val_mae: 0.1866
Epoch 5/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1781 - mae: 0.2775 - val_loss: 0.0630 - val_mae: 0.1696
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1657 - mae: 0.2677 - val_loss: 0.0784 - val_mae: 0.1704
Epoch 7/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1596 - mae: 0.2606 - val_loss: 0.1107 - val_mae: 0.1921
Epoch 8/100
1109/1109 [==============================] 

In [16]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 1ms/step


In [17]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.016103059974262236


# ReLU MSE

In [18]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 320)               11520     
                                                                 
 activation_3 (Activation)   (None, 320)               0         
                                                                 
 batch_normalization_9 (Batc  (None, 320)              1280      
 hNormalization)                                                 
                                                                 
 dropout_6 (Dropout)         (None, 320)               0         
                                                                 
 dense_13 (Dense)            (None, 128)               41088     
                                                                 
 activation_4 (Activation)   (None, 128)               0         
                                                      

In [19]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 6s 4ms/step - loss: 0.5093 - mse: 0.5093 - val_loss: 0.2201 - val_mse: 0.2201
Epoch 2/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.2867 - mse: 0.2867 - val_loss: 0.1446 - val_mse: 0.1446
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2310 - mse: 0.2310 - val_loss: 0.0849 - val_mse: 0.0849
Epoch 4/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2029 - mse: 0.2029 - val_loss: 0.0861 - val_mse: 0.0861
Epoch 5/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1800 - mse: 0.1800 - val_loss: 0.0610 - val_mse: 0.0610
Epoch 6/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1708 - mse: 0.1708 - val_loss: 0.0819 - val_mse: 0.0819
Epoch 7/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1623 - mse: 0.1623 - val_loss: 0.0841 - val_mse: 0.0841
Epoch 8/100
1109/1109 [==============================] 

In [20]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [21]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.014773179683697521


# ELU MAE

In [22]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(ELU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(ELU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(ELU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 320)               11520     
                                                                 
 elu (ELU)                   (None, 320)               0         
                                                                 
 batch_normalization_12 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_8 (Dropout)         (None, 320)               0         
                                                                 
 dense_17 (Dense)            (None, 128)               41088     
                                                                 
 elu_1 (ELU)                 (None, 128)               0         
                                                      

In [23]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 6s 4ms/step - loss: 0.4467 - mae: 0.3937 - val_loss: 0.1595 - val_mae: 0.2226
Epoch 2/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2646 - mae: 0.3154 - val_loss: 0.1312 - val_mae: 0.1866
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2089 - mae: 0.2861 - val_loss: 0.1041 - val_mae: 0.1964
Epoch 4/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1903 - mae: 0.2812 - val_loss: 0.0734 - val_mae: 0.1855
Epoch 5/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1716 - mae: 0.2749 - val_loss: 0.0539 - val_mae: 0.1720
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1581 - mae: 0.2664 - val_loss: 0.0509 - val_mae: 0.1545
Epoch 7/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1456 - mae: 0.2587 - val_loss: 0.0861 - val_mae: 0.1943
Epoch 8/100
1109/1109 [==============================] 

In [24]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [25]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.011249610856637704


# ELU MSE

In [26]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(ELU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(ELU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(ELU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 320)               11520     
                                                                 
 elu_3 (ELU)                 (None, 320)               0         
                                                                 
 batch_normalization_15 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_10 (Dropout)        (None, 320)               0         
                                                                 
 dense_21 (Dense)            (None, 128)               41088     
                                                                 
 elu_4 (ELU)                 (None, 128)               0         
                                                      

In [27]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 6s 4ms/step - loss: 0.4835 - mse: 0.4835 - val_loss: 0.1658 - val_mse: 0.1658
Epoch 2/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2810 - mse: 0.2810 - val_loss: 0.1259 - val_mse: 0.1259
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2140 - mse: 0.2140 - val_loss: 0.0853 - val_mse: 0.0853
Epoch 4/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1937 - mse: 0.1937 - val_loss: 0.1001 - val_mse: 0.1001
Epoch 5/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1717 - mse: 0.1717 - val_loss: 0.0568 - val_mse: 0.0568
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1559 - mse: 0.1559 - val_loss: 0.0569 - val_mse: 0.0569
Epoch 7/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1442 - mse: 0.1442 - val_loss: 0.0476 - val_mse: 0.0476
Epoch 8/100
1109/1109 [==============================] 

In [28]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [29]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.008644604061484816


# PReLU MAE

In [30]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(PReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 320)               11520     
                                                                 
 p_re_lu (PReLU)             (None, 320)               320       
                                                                 
 batch_normalization_18 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_12 (Dropout)        (None, 320)               0         
                                                                 
 dense_25 (Dense)            (None, 128)               41088     
                                                                 
 p_re_lu_1 (PReLU)           (None, 128)               128       
                                                      

In [31]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 7s 5ms/step - loss: 0.5683 - mae: 0.4531 - val_loss: 0.2304 - val_mae: 0.2257
Epoch 2/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.3160 - mae: 0.3176 - val_loss: 0.1142 - val_mae: 0.2005
Epoch 3/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.2424 - mae: 0.2954 - val_loss: 0.0762 - val_mae: 0.1887
Epoch 4/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1974 - mae: 0.2839 - val_loss: 0.0709 - val_mae: 0.1853
Epoch 5/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1687 - mae: 0.2722 - val_loss: 0.0612 - val_mae: 0.1740
Epoch 6/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1571 - mae: 0.2627 - val_loss: 0.0679 - val_mae: 0.1636
Epoch 7/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1521 - mae: 0.2618 - val_loss: 0.0720 - val_mae: 0.1788
Epoch 8/100
1109/1109 [==============================] 

In [32]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [33]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.01389751741720675


# PReLU MSE

In [34]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(PReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 320)               11520     
                                                                 
 p_re_lu_3 (PReLU)           (None, 320)               320       
                                                                 
 batch_normalization_21 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_14 (Dropout)        (None, 320)               0         
                                                                 
 dense_29 (Dense)            (None, 128)               41088     
                                                                 
 p_re_lu_4 (PReLU)           (None, 128)               128       
                                                      

In [35]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 7s 5ms/step - loss: 0.5348 - mse: 0.5348 - val_loss: 0.2254 - val_mse: 0.2254
Epoch 2/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.3146 - mse: 0.3146 - val_loss: 0.1556 - val_mse: 0.1556
Epoch 3/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.2494 - mse: 0.2494 - val_loss: 0.0909 - val_mse: 0.0909
Epoch 4/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.2027 - mse: 0.2027 - val_loss: 0.0648 - val_mse: 0.0648
Epoch 5/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1861 - mse: 0.1861 - val_loss: 0.0839 - val_mse: 0.0839
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1624 - mse: 0.1624 - val_loss: 0.0697 - val_mse: 0.0697
Epoch 7/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1425 - mse: 0.1425 - val_loss: 0.0677 - val_mse: 0.0677
Epoch 8/100
1109/1109 [==============================] 

In [36]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [37]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.009456420035341659


# Part II

# 4 Layer

In [38]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 320)               0         
                                                                 
 batch_normalization_24 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_16 (Dropout)        (None, 320)               0         
                                                                 
 dense_33 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 128)               0         
                                                      

In [39]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 6s 4ms/step - loss: 0.4473 - mse: 0.4473 - val_loss: 0.1735 - val_mse: 0.1735
Epoch 2/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2669 - mse: 0.2669 - val_loss: 0.1218 - val_mse: 0.1218
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2185 - mse: 0.2185 - val_loss: 0.0941 - val_mse: 0.0941
Epoch 4/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1828 - mse: 0.1828 - val_loss: 0.0584 - val_mse: 0.0584
Epoch 5/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1762 - mse: 0.1762 - val_loss: 0.1032 - val_mse: 0.1032
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1585 - mse: 0.1585 - val_loss: 0.1164 - val_mse: 0.1164
Epoch 7/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1556 - mse: 0.1556 - val_loss: 0.0638 - val_mse: 0.0638
Epoch 8/100
1109/1109 [==============================] 

In [40]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [41]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.009444589155674442


# 5 Layer

In [42]:
model = Sequential()

model.add(Dense(512, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 512)               18432     
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_27 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dropout_18 (Dropout)        (None, 512)               0         
                                                                 
 dense_37 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 256)               0         
                                                      

In [43]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 8s 5ms/step - loss: 0.5290 - mse: 0.5290 - val_loss: 0.2524 - val_mse: 0.2524
Epoch 2/100
1109/1109 [==============================] - 6s 5ms/step - loss: 0.3274 - mse: 0.3274 - val_loss: 0.1715 - val_mse: 0.1715
Epoch 3/100
1109/1109 [==============================] - 6s 5ms/step - loss: 0.2398 - mse: 0.2398 - val_loss: 0.1413 - val_mse: 0.1413
Epoch 4/100
1109/1109 [==============================] - 6s 5ms/step - loss: 0.2106 - mse: 0.2106 - val_loss: 0.1134 - val_mse: 0.1134
Epoch 5/100
1109/1109 [==============================] - 6s 5ms/step - loss: 0.1937 - mse: 0.1937 - val_loss: 0.0822 - val_mse: 0.0822
Epoch 6/100
1109/1109 [==============================] - 6s 5ms/step - loss: 0.1868 - mse: 0.1868 - val_loss: 0.0935 - val_mse: 0.0935
Epoch 7/100
1109/1109 [==============================] - 6s 5ms/step - loss: 0.1558 - mse: 0.1558 - val_loss: 0.0731 - val_mse: 0.0731
Epoch 8/100
1109/1109 [==============================] 

In [44]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [45]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.008924850422356695


# 6 Layer

In [46]:
model = Sequential()

model.add(Dense(512, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())


model.add(Dense(32))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_41 (Dense)            (None, 512)               18432     
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 512)               0         
                                                                 
 batch_normalization_31 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dropout_21 (Dropout)        (None, 512)               0         
                                                                 
 dense_42 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_14 (LeakyReLU)  (None, 256)               0         
                                                     

In [47]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 9s 6ms/step - loss: 0.4575 - mse: 0.4575 - val_loss: 0.2342 - val_mse: 0.2342
Epoch 2/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.2756 - mse: 0.2756 - val_loss: 0.5983 - val_mse: 0.5983
Epoch 3/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.2204 - mse: 0.2204 - val_loss: 0.1185 - val_mse: 0.1185
Epoch 4/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.1864 - mse: 0.1864 - val_loss: 0.0787 - val_mse: 0.0787
Epoch 5/100
1109/1109 [==============================] - 6s 6ms/step - loss: 0.1684 - mse: 0.1684 - val_loss: 0.0776 - val_mse: 0.0776
Epoch 6/100
1109/1109 [==============================] - 6s 6ms/step - loss: 0.1411 - mse: 0.1411 - val_loss: 0.0652 - val_mse: 0.0652
Epoch 7/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.1394 - mse: 0.1394 - val_loss: 0.0825 - val_mse: 0.0825
Epoch 8/100
1109/1109 [==============================] 

In [48]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 2s 2ms/step


In [49]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.009719647824346747


# 7 Layer

In [50]:
model = Sequential()

model.add(Dense(512, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())


model.add(Dense(32))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(16))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_47 (Dense)            (None, 512)               18432     
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 512)               0         
                                                                 
 batch_normalization_36 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dropout_24 (Dropout)        (None, 512)               0         
                                                                 
 dense_48 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_19 (LeakyReLU)  (None, 256)               0         
                                                     

In [51]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 10s 6ms/step - loss: 0.4289 - mse: 0.4289 - val_loss: 0.2426 - val_mse: 0.2426
Epoch 2/100
1109/1109 [==============================] - 6s 6ms/step - loss: 0.2767 - mse: 0.2767 - val_loss: 0.1257 - val_mse: 0.1257
Epoch 3/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.2324 - mse: 0.2324 - val_loss: 0.0816 - val_mse: 0.0816
Epoch 4/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.1834 - mse: 0.1834 - val_loss: 0.0829 - val_mse: 0.0829
Epoch 5/100
1109/1109 [==============================] - 6s 6ms/step - loss: 0.1741 - mse: 0.1741 - val_loss: 0.0615 - val_mse: 0.0615
Epoch 6/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.1508 - mse: 0.1508 - val_loss: 0.0651 - val_mse: 0.0651
Epoch 7/100
1109/1109 [==============================] - 6s 6ms/step - loss: 0.1494 - mse: 0.1494 - val_loss: 0.0797 - val_mse: 0.0797
Epoch 8/100
1109/1109 [==============================]

In [52]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 2s 2ms/step


In [53]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.009356605106137483


# PART III

# Adam

In [54]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_54 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_24 (LeakyReLU)  (None, 320)               0         
                                                                 
 batch_normalization_42 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_27 (Dropout)        (None, 320)               0         
                                                                 
 dense_55 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_25 (LeakyReLU)  (None, 128)               0         
                                                     

In [55]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 6s 4ms/step - loss: 0.4981 - mse: 0.4981 - val_loss: 0.2620 - val_mse: 0.2620
Epoch 2/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.2672 - mse: 0.2672 - val_loss: 0.1252 - val_mse: 0.1252
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2199 - mse: 0.2199 - val_loss: 0.0869 - val_mse: 0.0869
Epoch 4/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1897 - mse: 0.1897 - val_loss: 0.0826 - val_mse: 0.0826
Epoch 5/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1741 - mse: 0.1741 - val_loss: 0.0853 - val_mse: 0.0853
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1591 - mse: 0.1591 - val_loss: 0.1047 - val_mse: 0.1047
Epoch 7/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.1452 - mse: 0.1452 - val_loss: 0.0730 - val_mse: 0.0730
Epoch 8/100
1109/1109 [==============================] 

In [56]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [57]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.007904961908444882


# SGD

In [58]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = SGD(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_58 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_27 (LeakyReLU)  (None, 320)               0         
                                                                 
 batch_normalization_45 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_29 (Dropout)        (None, 320)               0         
                                                                 
 dense_59 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_28 (LeakyReLU)  (None, 128)               0         
                                                     

In [59]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 6s 5ms/step - loss: 0.7932 - mse: 0.7932 - val_loss: 0.4055 - val_mse: 0.4055
Epoch 2/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.5224 - mse: 0.5224 - val_loss: 0.3601 - val_mse: 0.3601
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.4644 - mse: 0.4644 - val_loss: 0.3747 - val_mse: 0.3747
Epoch 4/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.4340 - mse: 0.4340 - val_loss: 0.3538 - val_mse: 0.3538
Epoch 5/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.4182 - mse: 0.4182 - val_loss: 0.3466 - val_mse: 0.3466
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.4060 - mse: 0.4060 - val_loss: 0.3405 - val_mse: 0.3405
Epoch 7/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.3981 - mse: 0.3981 - val_loss: 0.3449 - val_mse: 0.3449
Epoch 8/100
1109/1109 [==============================] 

In [60]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [61]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.014555960271539544


# RMSprop

In [62]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = RMSprop(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_62 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_30 (LeakyReLU)  (None, 320)               0         
                                                                 
 batch_normalization_48 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_31 (Dropout)        (None, 320)               0         
                                                                 
 dense_63 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_31 (LeakyReLU)  (None, 128)               0         
                                                     

In [63]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 6s 5ms/step - loss: 0.4550 - mse: 0.4550 - val_loss: 0.1970 - val_mse: 0.1970
Epoch 2/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.2702 - mse: 0.2702 - val_loss: 0.1662 - val_mse: 0.1662
Epoch 3/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.2189 - mse: 0.2189 - val_loss: 0.1610 - val_mse: 0.1610
Epoch 4/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1965 - mse: 0.1965 - val_loss: 0.1145 - val_mse: 0.1145
Epoch 5/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1858 - mse: 0.1858 - val_loss: 0.1193 - val_mse: 0.1193
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1712 - mse: 0.1712 - val_loss: 0.0875 - val_mse: 0.0875
Epoch 7/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1623 - mse: 0.1623 - val_loss: 0.1849 - val_mse: 0.1849
Epoch 8/100
1109/1109 [==============================] 

In [64]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [65]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.009724310329524097


# Adagrad

In [66]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adagrad(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_66 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_33 (LeakyReLU)  (None, 320)               0         
                                                                 
 batch_normalization_51 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_33 (Dropout)        (None, 320)               0         
                                                                 
 dense_67 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_34 (LeakyReLU)  (None, 128)               0         
                                                     

In [67]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 6s 4ms/step - loss: 0.9313 - mse: 0.9313 - val_loss: 0.4618 - val_mse: 0.4618
Epoch 2/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.6609 - mse: 0.6609 - val_loss: 0.4272 - val_mse: 0.4272
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.6005 - mse: 0.6005 - val_loss: 0.4486 - val_mse: 0.4486
Epoch 4/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.5752 - mse: 0.5752 - val_loss: 0.4224 - val_mse: 0.4224
Epoch 5/100
1109/1109 [==============================] - 4s 4ms/step - loss: 0.5422 - mse: 0.5422 - val_loss: 0.4066 - val_mse: 0.4066
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.5364 - mse: 0.5364 - val_loss: 0.3845 - val_mse: 0.3845
Epoch 7/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.5179 - mse: 0.5179 - val_loss: 0.4025 - val_mse: 0.4025
Epoch 8/100
1109/1109 [==============================] 

1109/1109 [==============================] - 5s 4ms/step - loss: 0.3747 - mse: 0.3747 - val_loss: 0.3121 - val_mse: 0.3121
Epoch 62/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.3762 - mse: 0.3762 - val_loss: 0.3119 - val_mse: 0.3119
Epoch 63/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.3777 - mse: 0.3777 - val_loss: 0.2987 - val_mse: 0.2987
Epoch 64/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.3762 - mse: 0.3762 - val_loss: 0.2975 - val_mse: 0.2975


In [68]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [69]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.01976237682808322


# Adadelta

In [70]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adadelta(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_70 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_36 (LeakyReLU)  (None, 320)               0         
                                                                 
 batch_normalization_54 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_35 (Dropout)        (None, 320)               0         
                                                                 
 dense_71 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_37 (LeakyReLU)  (None, 128)               0         
                                                     

In [71]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 7s 5ms/step - loss: 2.6771 - mse: 2.6771 - val_loss: 1.0360 - val_mse: 1.0360
Epoch 2/100
1109/1109 [==============================] - 5s 4ms/step - loss: 2.3390 - mse: 2.3390 - val_loss: 0.8536 - val_mse: 0.8536
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 2.1639 - mse: 2.1639 - val_loss: 0.7504 - val_mse: 0.7504
Epoch 4/100
1109/1109 [==============================] - 5s 4ms/step - loss: 2.0254 - mse: 2.0254 - val_loss: 0.6837 - val_mse: 0.6837
Epoch 5/100
1109/1109 [==============================] - 5s 4ms/step - loss: 1.9187 - mse: 1.9187 - val_loss: 0.6595 - val_mse: 0.6595
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 1.7921 - mse: 1.7921 - val_loss: 0.6210 - val_mse: 0.6210
Epoch 7/100
1109/1109 [==============================] - 5s 4ms/step - loss: 1.7307 - mse: 1.7307 - val_loss: 0.5977 - val_mse: 0.5977
Epoch 8/100
1109/1109 [==============================] 

1109/1109 [==============================] - 5s 4ms/step - loss: 0.7669 - mse: 0.7669 - val_loss: 0.3588 - val_mse: 0.3588
Epoch 62/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.7698 - mse: 0.7698 - val_loss: 0.3556 - val_mse: 0.3556
Epoch 63/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.7723 - mse: 0.7723 - val_loss: 0.3549 - val_mse: 0.3549
Epoch 64/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.7703 - mse: 0.7703 - val_loss: 0.3543 - val_mse: 0.3543
Epoch 65/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.7552 - mse: 0.7552 - val_loss: 0.3544 - val_mse: 0.3544
Epoch 66/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.7595 - mse: 0.7595 - val_loss: 0.3541 - val_mse: 0.3541
Epoch 67/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.7719 - mse: 0.7719 - val_loss: 0.3534 - val_mse: 0.3534
Epoch 68/100
1109/1109 [==============================] - 5s 

In [72]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [73]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.02219015474013301


# Adamax

In [74]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adamax(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_74 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_39 (LeakyReLU)  (None, 320)               0         
                                                                 
 batch_normalization_57 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_37 (Dropout)        (None, 320)               0         
                                                                 
 dense_75 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_40 (LeakyReLU)  (None, 128)               0         
                                                     

In [75]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 7s 4ms/step - loss: 0.6583 - mse: 0.6583 - val_loss: 0.3071 - val_mse: 0.3071
Epoch 2/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.4465 - mse: 0.4465 - val_loss: 0.2589 - val_mse: 0.2589
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.3788 - mse: 0.3788 - val_loss: 0.2147 - val_mse: 0.2147
Epoch 4/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.3400 - mse: 0.3400 - val_loss: 0.1809 - val_mse: 0.1809
Epoch 5/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.3039 - mse: 0.3039 - val_loss: 0.1578 - val_mse: 0.1578
Epoch 6/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.2827 - mse: 0.2827 - val_loss: 0.1424 - val_mse: 0.1424
Epoch 7/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2522 - mse: 0.2522 - val_loss: 0.1111 - val_mse: 0.1111
Epoch 8/100
1109/1109 [==============================] 

In [76]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 2s 2ms/step


In [77]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.008548405000241292


# Nadam

In [78]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Nadam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_78 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_42 (LeakyReLU)  (None, 320)               0         
                                                                 
 batch_normalization_60 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_39 (Dropout)        (None, 320)               0         
                                                                 
 dense_79 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_43 (LeakyReLU)  (None, 128)               0         
                                                     

In [79]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 10s 5ms/step - loss: 0.5091 - mse: 0.5091 - val_loss: 0.3095 - val_mse: 0.3095
Epoch 2/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.2852 - mse: 0.2852 - val_loss: 0.2223 - val_mse: 0.2223
Epoch 3/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.2118 - mse: 0.2118 - val_loss: 0.0916 - val_mse: 0.0916
Epoch 4/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1873 - mse: 0.1873 - val_loss: 0.0909 - val_mse: 0.0909
Epoch 5/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1666 - mse: 0.1666 - val_loss: 0.1018 - val_mse: 0.1018
Epoch 6/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1607 - mse: 0.1607 - val_loss: 0.0803 - val_mse: 0.0803
Epoch 7/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1396 - mse: 0.1396 - val_loss: 0.1157 - val_mse: 0.1157
Epoch 8/100
1109/1109 [==============================]

In [80]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [81]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.00785058930332007


# Ftrl

In [82]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Ftrl(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_82 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_45 (LeakyReLU)  (None, 320)               0         
                                                                 
 batch_normalization_63 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_41 (Dropout)        (None, 320)               0         
                                                                 
 dense_83 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_46 (LeakyReLU)  (None, 128)               0         
                                                     

In [83]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 8s 5ms/step - loss: 0.5191 - mse: 0.5191 - val_loss: 0.2953 - val_mse: 0.2953
Epoch 2/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.4062 - mse: 0.4062 - val_loss: 0.2612 - val_mse: 0.2612
Epoch 3/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.3436 - mse: 0.3436 - val_loss: 0.2401 - val_mse: 0.2401
Epoch 4/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.3196 - mse: 0.3196 - val_loss: 0.2280 - val_mse: 0.2280
Epoch 5/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.2840 - mse: 0.2840 - val_loss: 0.2122 - val_mse: 0.2122
Epoch 6/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.2646 - mse: 0.2646 - val_loss: 0.1868 - val_mse: 0.1868
Epoch 7/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.2580 - mse: 0.2580 - val_loss: 0.1863 - val_mse: 0.1863
Epoch 8/100
1109/1109 [==============================] 

In [84]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [85]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.011186004636446643


# PART IV

# Different Epochs

In [86]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_86 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_48 (LeakyReLU)  (None, 320)               0         
                                                                 
 batch_normalization_66 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_43 (Dropout)        (None, 320)               0         
                                                                 
 dense_87 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_49 (LeakyReLU)  (None, 128)               0         
                                                     

In [87]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=200, batch_size=32, callbacks=[early_stopping])

Epoch 1/200
1109/1109 [==============================] - 8s 5ms/step - loss: 0.4710 - mse: 0.4710 - val_loss: 0.2148 - val_mse: 0.2148
Epoch 2/200
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2770 - mse: 0.2770 - val_loss: 0.1239 - val_mse: 0.1239
Epoch 3/200
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2272 - mse: 0.2272 - val_loss: 0.1114 - val_mse: 0.1114
Epoch 4/200
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1872 - mse: 0.1872 - val_loss: 0.1603 - val_mse: 0.1603
Epoch 5/200
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1797 - mse: 0.1797 - val_loss: 0.0657 - val_mse: 0.0657
Epoch 6/200
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1698 - mse: 0.1698 - val_loss: 0.1168 - val_mse: 0.1168
Epoch 7/200
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1549 - mse: 0.1549 - val_loss: 0.0984 - val_mse: 0.0984
Epoch 8/200
1109/1109 [==============================] 

In [88]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [89]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.008221715096435288


#  PART V

# Normal

In [90]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_90 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_51 (LeakyReLU)  (None, 320)               0         
                                                                 
 batch_normalization_69 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_45 (Dropout)        (None, 320)               0         
                                                                 
 dense_91 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_52 (LeakyReLU)  (None, 128)               0         
                                                     

In [91]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 7s 5ms/step - loss: 0.4512 - mse: 0.4512 - val_loss: 0.1535 - val_mse: 0.1535
Epoch 2/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2631 - mse: 0.2631 - val_loss: 0.2005 - val_mse: 0.2005
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2039 - mse: 0.2039 - val_loss: 0.1353 - val_mse: 0.1353
Epoch 4/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1930 - mse: 0.1930 - val_loss: 0.1141 - val_mse: 0.1141
Epoch 5/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1701 - mse: 0.1701 - val_loss: 0.1378 - val_mse: 0.1378
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1600 - mse: 0.1600 - val_loss: 0.1018 - val_mse: 0.1018
Epoch 7/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1505 - mse: 0.1505 - val_loss: 0.0813 - val_mse: 0.0813
Epoch 8/100
1109/1109 [==============================] 

In [92]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [93]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.009073516902235047


#  Add L2 Regularization

In [94]:
from keras import regularizers

model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1], kernel_regularizer=regularizers.l2(0.01)))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, kernel_regularizer=regularizers.l2(0.01)))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, kernel_regularizer=regularizers.l2(0.01)))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_94 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_54 (LeakyReLU)  (None, 320)               0         
                                                                 
 batch_normalization_72 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_47 (Dropout)        (None, 320)               0         
                                                                 
 dense_95 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_55 (LeakyReLU)  (None, 128)               0         
                                                     

In [95]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=200, batch_size=32, callbacks=[early_stopping])

Epoch 1/200
1109/1109 [==============================] - 7s 5ms/step - loss: 2.2786 - mse: 0.4799 - val_loss: 1.0550 - val_mse: 0.2669
Epoch 2/200
1109/1109 [==============================] - 5s 4ms/step - loss: 0.8321 - mse: 0.3274 - val_loss: 0.9067 - val_mse: 0.5789
Epoch 3/200
1109/1109 [==============================] - 5s 4ms/step - loss: 0.5337 - mse: 0.2827 - val_loss: 0.4026 - val_mse: 0.1810
Epoch 4/200
1109/1109 [==============================] - 5s 4ms/step - loss: 0.4836 - mse: 0.2788 - val_loss: 0.3735 - val_mse: 0.2144
Epoch 5/200
1109/1109 [==============================] - 5s 4ms/step - loss: 0.4474 - mse: 0.2660 - val_loss: 0.2982 - val_mse: 0.1101
Epoch 6/200
1109/1109 [==============================] - 5s 4ms/step - loss: 0.4372 - mse: 0.2593 - val_loss: 0.4812 - val_mse: 0.3138
Epoch 7/200
1109/1109 [==============================] - 5s 4ms/step - loss: 0.4477 - mse: 0.2656 - val_loss: 0.3075 - val_mse: 0.1294
Epoch 8/200
1109/1109 [==============================] 

In [96]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [97]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.011268913794814144


# Part VI

#  Normal


In [98]:
model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_98 (Dense)            (None, 320)               11520     
                                                                 
 leaky_re_lu_57 (LeakyReLU)  (None, 320)               0         
                                                                 
 batch_normalization_75 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_49 (Dropout)        (None, 320)               0         
                                                                 
 dense_99 (Dense)            (None, 128)               41088     
                                                                 
 leaky_re_lu_58 (LeakyReLU)  (None, 128)               0         
                                                     

In [99]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=200, batch_size=32, callbacks=[early_stopping])

Epoch 1/200
1109/1109 [==============================] - 8s 5ms/step - loss: 0.5023 - mse: 0.5023 - val_loss: 0.2265 - val_mse: 0.2265
Epoch 2/200
1109/1109 [==============================] - 5s 5ms/step - loss: 0.2797 - mse: 0.2797 - val_loss: 0.1232 - val_mse: 0.1232
Epoch 3/200
1109/1109 [==============================] - 5s 5ms/step - loss: 0.2210 - mse: 0.2210 - val_loss: 0.1158 - val_mse: 0.1158
Epoch 4/200
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1984 - mse: 0.1984 - val_loss: 0.0848 - val_mse: 0.0848
Epoch 5/200
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1764 - mse: 0.1764 - val_loss: 0.0676 - val_mse: 0.0676
Epoch 6/200
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1681 - mse: 0.1681 - val_loss: 0.1970 - val_mse: 0.1970
Epoch 7/200
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1632 - mse: 0.1632 - val_loss: 0.0700 - val_mse: 0.0700
Epoch 8/200
1109/1109 [==============================] 

In [100]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [101]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.009057954541687771


#  Add Kernel initializer

In [102]:
from keras.initializers import he_normal

model = Sequential()

model.add(Dense(320, input_dim=X_train_scaled.shape[1], kernel_initializer=he_normal()))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, kernel_initializer=he_normal()))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, kernel_initializer=he_normal()))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_102 (Dense)           (None, 320)               11520     
                                                                 
 leaky_re_lu_60 (LeakyReLU)  (None, 320)               0         
                                                                 
 batch_normalization_78 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dropout_51 (Dropout)        (None, 320)               0         
                                                                 
 dense_103 (Dense)           (None, 128)               41088     
                                                                 
 leaky_re_lu_61 (LeakyReLU)  (None, 128)               0         
                                                     

In [103]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 7s 5ms/step - loss: 0.4826 - mse: 0.4826 - val_loss: 0.2848 - val_mse: 0.2848
Epoch 2/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.2961 - mse: 0.2961 - val_loss: 0.2046 - val_mse: 0.2046
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.2416 - mse: 0.2416 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 4/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1942 - mse: 0.1942 - val_loss: 0.1070 - val_mse: 0.1070
Epoch 5/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1803 - mse: 0.1803 - val_loss: 0.0756 - val_mse: 0.0756
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1595 - mse: 0.1595 - val_loss: 0.0629 - val_mse: 0.0629
Epoch 7/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1485 - mse: 0.1485 - val_loss: 0.0691 - val_mse: 0.0691
Epoch 8/100
1109/1109 [==============================] 

In [104]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [105]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.007317440521250598


# Part VII

In [106]:
from keras.initializers import he_normal

model = Sequential()

model.add(Dense(340, input_dim=X_train_scaled.shape[1], kernel_initializer=he_normal()))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(200, kernel_initializer=he_normal()))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(32, kernel_initializer=he_normal()))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_106 (Dense)           (None, 340)               12240     
                                                                 
 leaky_re_lu_63 (LeakyReLU)  (None, 340)               0         
                                                                 
 batch_normalization_81 (Bat  (None, 340)              1360      
 chNormalization)                                                
                                                                 
 dropout_53 (Dropout)        (None, 340)               0         
                                                                 
 dense_107 (Dense)           (None, 200)               68200     
                                                                 
 leaky_re_lu_64 (LeakyReLU)  (None, 200)               0         
                                                     

In [107]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 7s 5ms/step - loss: 0.4960 - mse: 0.4960 - val_loss: 0.2546 - val_mse: 0.2546
Epoch 2/100
1109/1109 [==============================] - 6s 5ms/step - loss: 0.2852 - mse: 0.2852 - val_loss: 0.6540 - val_mse: 0.6540
Epoch 3/100
1109/1109 [==============================] - 6s 5ms/step - loss: 0.2271 - mse: 0.2271 - val_loss: 0.2248 - val_mse: 0.2248
Epoch 4/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1927 - mse: 0.1927 - val_loss: 0.1118 - val_mse: 0.1118
Epoch 5/100
1109/1109 [==============================] - 6s 5ms/step - loss: 0.1688 - mse: 0.1688 - val_loss: 0.0672 - val_mse: 0.0672
Epoch 6/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1565 - mse: 0.1565 - val_loss: 0.0811 - val_mse: 0.0811
Epoch 7/100
1109/1109 [==============================] - 5s 5ms/step - loss: 0.1486 - mse: 0.1486 - val_loss: 0.1009 - val_mse: 0.1009
Epoch 8/100
1109/1109 [==============================] 

In [108]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [109]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.007657155353689527


In [110]:
model = Sequential()

model.add(Dense(96, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.30000000000000004))

model.add(Dense(384))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.30000000000000004))

model.add(Dense(32))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_110 (Dense)           (None, 96)                3456      
                                                                 
 leaky_re_lu_66 (LeakyReLU)  (None, 96)                0         
                                                                 
 batch_normalization_84 (Bat  (None, 96)               384       
 chNormalization)                                                
                                                                 
 dropout_55 (Dropout)        (None, 96)                0         
                                                                 
 dense_111 (Dense)           (None, 384)               37248     
                                                                 
 leaky_re_lu_67 (LeakyReLU)  (None, 384)               0         
                                                     

In [111]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 7s 5ms/step - loss: 0.3656 - mse: 0.3656 - val_loss: 0.1561 - val_mse: 0.1561
Epoch 2/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1966 - mse: 0.1966 - val_loss: 0.0917 - val_mse: 0.0917
Epoch 3/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1612 - mse: 0.1612 - val_loss: 0.0624 - val_mse: 0.0624
Epoch 4/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1401 - mse: 0.1401 - val_loss: 0.0985 - val_mse: 0.0985
Epoch 5/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1311 - mse: 0.1311 - val_loss: 0.2541 - val_mse: 0.2541
Epoch 6/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1185 - mse: 0.1185 - val_loss: 0.0466 - val_mse: 0.0466
Epoch 7/100
1109/1109 [==============================] - 5s 4ms/step - loss: 0.1201 - mse: 0.1201 - val_loss: 0.0732 - val_mse: 0.0732
Epoch 8/100
1109/1109 [==============================] 

In [112]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)

675/675 [==============================] - 1s 2ms/step


In [113]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.007131564824568299
